In [824]:
%run data_plot.ipynb

In [825]:
'''
initialize the variables for storing possible routes and the 
best route for visualization and saving in a solutions file
'''
aco_data = {'best_path': {},
            'possible_paths': {}}
solutions = [{}]

In [826]:
trail_data = {}
ant_data = {}

In [827]:
def init_ants(origin, coords_data, ants_count, ant_data):
    """
    init_ants(origin, coords_data, ants_count, ant_data)
    initilizes the ants on the search space. This is done by setting the ants
    on random nodes including the origin node. For each node, 2 ants are 
    initialized and for each of them on a single node, the destination is set to
    one of the base stations. the other ants on nodes otther than the origin node
    serve as best path discovery ants as they will deposit pheromone on good paths 
    and this will help our ant of interest in deciding a suitable path to its destination
    origin: the node (sensor) of interest whose best path to a base station we seek
    coords_data: the x_y_data of the sensors in the search space
    ants_count: number of ants to be initiated
    ant_data: the global variable containing the ants data
    returns: the ants' data with updated values
    """
    ants_count = math.ceil(ants_count / 2)
    dest_nodes = [x_y_base_station_1[0], x_y_base_station_2[0]]
    init_nodes = choice(np.array([i for i in coords_data.keys() if i not in dest_nodes]), ants_count, replace=False)
    # one of the origin nodes has to be the defined origin
    if origin not in init_nodes:
        init_nodes[-1] = origin
    i = 1
    for node in init_nodes:
        for dest_node in dest_nodes:
            ant_data[i] = {'start_node': node, 'destination_node': dest_node, 'paths': []}
            i += 1

    return ant_data

In [828]:
#init_ants(15, x_y_data, 20, ant_data)

In [829]:
def init_pheromone(coords_data, trail_data):
    """
    init_pheromone(coords_data, trail_data)
    function that initializes pheromone on the various trails in the search space.
    the amount of pheromone initialized is random and is a number between 0 and 1.
    the pheromone information is input in the trail_data dictionary
    coords_data: the x, y coordinate data
    trail_data: the 'trail_data' global variable where pheromone information will be added
    returns: the updated 'trail_data' information
    """
    for data_row in coords_data.keys():
        # init an array. will be a matrix with pheromone data
        trail_row = []
        for data_col in coords_data.keys():
            if data_row == data_col:
                trail_row.append(0)
            else:
                if data_col > data_row:
                    # initialize the pheromone with a random number between 0 and 1
                    trail_row.append(random.uniform(0, 1))
                else:
                    trail_row.append(trail_data[data_col][data_row - 1])
        trail_data[data_row] = trail_row

    return trail_data

In [830]:
#init_pheromone(x_y_data, trail_data)
#print(trail_data)

In [831]:
def calculate_route_costs(coords_data):
    """
    calculate_route_costs(coords_data)
    calculates the bandwidth between nodes and each other. This is done using 
    the calculate_distance() and convert_distance_bandwidth() functions of the 
    data_plot module and added to a 2D array
    coords_data: the x, y coordinate data
    returns: the calculated bandwidth information for the various paths
    """
    bandwidth_data = []

    for data_row in coords_data.keys():
        # var for storing all path bandwidths & latencies
        path_costs = []
        for data_col in coords_data.keys():
            if data_row != data_col:
                # get path distance
                path_distance = calculate_distance(coords_data[data_row], coords_data[data_col])
                #print(f"Node {data_row} to Node {data_col}: {path_distance}")
                # get bandwidth between node and its next node and append it to path bandwidths var
                path_bandwidth = convert_distance_bandwidth(path_distance)
                #if path_bandwidth == 0:
                    #absolute_path_cost = 0
                #else:
                    #absolute_path_cost = round(absolute_cost_function(path_bandwidth, node_to_node_latency), 3)
                path_costs.append(path_bandwidth)
            else:
                path_costs.append(0)
        bandwidth_data.append(path_costs)

    return bandwidth_data

In [832]:
#bandwidth_data = calculate_route_costs(x_y_data)
#print(bandwidth_data)

In [833]:
def traverse_nodes(ant_data, coords_data, bandwidth_data, trail_data, alpha = 0.6):
    """
    traverse_nodes(ant_data, coords_data, bandwidth_data, trail_data, alpha = 0.6)
    traversal of the initialized ants from their respective origin nodes to the destination
    node. This is done iteratively until the ant gets to the destination. Selection of the path
    to the destination is dictated by a probabilistic function that is determined by a combination 
    of the path cost and the pheromone quantity. As this happens, the ants trail is updated in the
    ants_data global variable
    ant_data: the global variable containing the ants data
    coords_data: the x, y coordinate data
    bandwidth_data: the data containing bandwidth information that was calculated in calculate_route_costs()
    trail_data: the 'trail_data' global variable with pheromone level information
    alpha (optional): weight used to balance the significance of the pheromone level and path cost in determining
    the probability of an ant choosing a path (default: 0.6)
    returns: nothing
    """
    for ant in ant_data.keys():
        current_node = ant_data[ant]['start_node']
        ant_data[ant]['paths'].append({'path': [current_node], 'end-to-end_transmission': 0, 'latency': 0, 'cost': 0})
        # if the current path is not destination node
        while current_node != ant_data[ant]['destination_node']:
            # loop through all nodes and calculate sum of probabilities
            sum_of_probabilities = 0
            path_latency = 0
            for node in coords_data.keys():
                if node != current_node:
                    '''
                    calculate end to end bandwidth. First get all visited 
                    node in the path from origin up until the current node
                    '''
                    current_path = ant_data[ant]['paths'][-1]['path']
                    path_bandwidths = []
                    for path in range(len(current_path) - 1):
                        path_bandwidths.append(bandwidth_data[current_path[path] - 1][current_path[path + 1] - 1])
                    bandwidth = bandwidth_data[current_node - 1][node - 1]
                    path_bandwidths.append(bandwidth)
                    # compute end-to-end bandwidth rate
                    end_to_end_bandwidth = min(path_bandwidths)
                    path_latency += node_to_node_latency
                    # for each node further from original node, increase latency
                    if end_to_end_bandwidth == 0:
                        absolute_path_cost = 0
                    else:
                        # calculate absolute cost
                        absolute_path_cost = round(absolute_cost_function(bandwidth_data[current_node - 1][node - 1], path_latency), 3)
                    sum_of_probabilities += (trail_data[current_node][node - 1] * alpha) + \
                    ((1 - alpha) * absolute_path_cost)
            path_probabilities = [(None, 0)]
            # loop through all nodes and calculate probabilities
            for node in coords_data.keys():
                # dont calculate probability of visiting itself
                if node != current_node:
                    '''
                    calculate end to end bandwidth. First get all visited 
                    node in the path from origin up until the current node
                    '''
                    current_path = ant_data[ant]['paths'][-1]['path']
                    path_bandwidths = []
                    for path in range(len(current_path) - 1):
                        path_bandwidths.append(bandwidth_data[current_path[path] - 1][current_path[path + 1] - 1])
                    bandwidth = bandwidth_data[current_node - 1][node - 1]
                    path_bandwidths.append(bandwidth)
                    # compute end-to-end bandwidth rate
                    end_to_end_bandwidth = min(path_bandwidths)
                    path_latency += node_to_node_latency
                    # for each node further from original node, increase latency
                    if end_to_end_bandwidth == 0:
                        absolute_path_cost = 0
                    else:
                        # calculate absolute cost
                        absolute_path_cost = round(absolute_cost_function(bandwidth_data[current_node - 1][node - 1], path_latency), 3)
                    probability = ((trail_data[current_node][node - 1] * alpha) + \
                    ((1 - alpha) * absolute_path_cost)) \
                    / sum_of_probabilities
                    # do a cummulative probability that will be used to determine next path based
                    path_probabilities.append((node, path_probabilities[-1][1] + probability))
            rand_selector = random.uniform(0, 1)
            number_of_probs = len(path_probabilities)
            for idx in range(len(path_probabilities) - 1):
                if rand_selector > path_probabilities[idx][1] and rand_selector < path_probabilities[idx + 1][1]:
                    current_node = path_probabilities[idx + 1][0]
                    ant_data[ant]['paths'][-1]['path'].append(current_node)
                    break
                if idx + 1 == number_of_probs:
                    # if rand is number extremely close to one, select the last index
                    current_node = path_probabilities[idx + 1][0]
                    ant_data[ant]['paths'][-1]['path'].append(current_node)
                 
    return 

In [834]:
#traverse_nodes(ant_data, 15, x_y_data, bandwidth_data, trail_data)
#print(ant_data)

In [835]:
def remove_path_loops(ant_data):
    """
    remove_path_loops(ant_data)
    function that removes loops as a result of recursively visited nodes.
    this is done by removing the segment between the revisited node for a 
    straight forward path from origin to base station. This done for every
    ant and the data is found in the ant_data global variable
    ant_data: ant data data containing the path followed from the origin to
    the destination
    returns: nothing
    """
    # perform the process for every ant
    for ant, data in ant_data.items():
        # deal with the latest path for every iteration
        latest_path = data['paths'][-1]['path']
        modified = True

        while modified:
            modified = False
            for idx in range(len(latest_path)):
                '''
                loop through all values in path and check if there is any repeating itself
                This is done by comparing numberson the left side of the path with those from
                the right side of the path. The outermost repitition is the only loop to be 
                considered e.g. in the path '4->5->1->3->1->8->5->6', the repitition 5, 5 is
                the one to be removed. No need of checking for the other inner ones
                '''
                last_idx = len(latest_path) - 1
                while idx < last_idx:
                    if latest_path[idx] == latest_path[last_idx]:
                        latest_path = latest_path[:idx] + latest_path[last_idx:]
                        modified = True
                        break
                    last_idx -= 1
                if modified:
                    break 
        ant_data[ant]['paths'][-1]['path'] = latest_path

    return

In [836]:
#remove_path_loops(ant_data)

In [837]:
#ant_data

In [838]:
def calculate_path_total_cost(ant_data, bandwidth_data):
    """
    calculate_path_total_cost(ant_data, bandwidth_data)
    function to calculate the end-to-end absolute path cost using the
    updated ant path information. This is done by checking the visited nodes
    and their various bandwith and latency costs to calculated an absolute 
    end-to-end path cost.
    ant_data: the ant data global variable with information on the paths the various 
    ants have followed
    bandwidth: information on the bandwidths between nodes and each other
    returns: nothing
    """
    for ant, data in ant_data.items():
        latest_path = data['paths'][-1]['path']
        path_cost = data['paths'][-1]['cost']
        path_latency = 0
        # deal with the latest path for every iteration
        if len(latest_path) > 1:
            path_bandwidths = []
            for path in range(len(latest_path) - 1):
                path_bandwidths.append(bandwidth_data[latest_path[path] - 1][latest_path[path + 1] - 1])
                path_latency += node_to_node_latency
            # compute end-to-end bandwidth rate
            end_to_end_bandwidth = min(path_bandwidths)
            if end_to_end_bandwidth == 0:
                absolute_path_cost = 0
            else:
                # calculate absolute cost
                absolute_path_cost = round(absolute_cost_function(bandwidth_data[latest_path[path] - 1][latest_path[path + 1] - 1], path_latency), 3)
            # update ant data with all metrics
            ant_data[ant]['paths'][-1]['cost'] = absolute_path_cost  
            ant_data[ant]['paths'][-1]['end-to-end_transmission'] = end_to_end_bandwidth
            ant_data[ant]['paths'][-1]['latency'] = path_latency

    return

In [839]:
#calculate_path_total_cost(ant_data, bandwidth_data)

In [840]:
#ant_data

In [841]:
def negative_feedback(evaporation_rate, trail_data):
    """
    negative_feedback(evaporation_rate, trail_data)
    function that implements evaporation of pheromone on the trail after each
    iteration (round of searching for path to destination from origin for the 
    various ants.
    evaporation_rate: the percentage of pheromone evaporation with the period of time
    trail_data: information in pheromone quantities on the trail
    returns: nothing
    """
    for node, pheromone_data in trail_data.items():
        for idx, pheromone in enumerate(pheromone_data):
            trail_data[node][idx] = (1 - evaporation_rate) * pheromone

    return

In [842]:
#print(trail_data)

In [843]:
#negative_feedback(0.45, trail_data)

In [844]:
#print(trail_data)

In [845]:
def positive_feedback(Q, ant_data, trail_data, bandwidth_data):
    """
    positive_feedback(Q, ant_data, trail_data, bandwidth_data)
    increases pheromone quantity for the paths followed by the ants. Each path
    followed by an ant has pheromone quantity equivalent to the number of ants
    that have visited that path added to the path. This amount is also dictated
    by Q with is used to compute the proportion of pheromone on the basis of the
    bandwidth of that path
    Q: weight used to reward a path that has been followed by an ant with pheromone
    ant_data: the ant data global variable with information on the paths the various 
    ants have followed
    trail_data: information in pheromone quantities on the trail
    bandwidth_data: information on the bandwidths between nodes and each other
    returns: nothing
    """
    for ant, data in ant_data.items():
        latest_path = data['paths'][-1]['path']

        if len(latest_path) > 1:
            for idx in range(len(latest_path) - 1):
                # increase pheromone for path in both directions [a -> b] == [b -> a]
                if bandwidth_data[latest_path[idx] - 1][latest_path[idx + 1] - 1] == 0:
                    trail_data[latest_path[idx]][latest_path[idx + 1] - 1] += 0
                    trail_data[latest_path[idx + 1]][latest_path[idx] - 1] += 0
                else:
                    trail_data[latest_path[idx]][latest_path[idx + 1] - 1] += Q / bandwidth_data[latest_path[idx] - 1][latest_path[idx + 1] - 1]
                    trail_data[latest_path[idx + 1]][latest_path[idx] - 1] += Q / bandwidth_data[latest_path[idx] - 1][latest_path[idx + 1] - 1]

    return

In [846]:
#positive_feedback(1, ant_data, trail_data, bandwidth_data)

In [847]:
#print(trail_data)

In [848]:
def get_aco_data(origin, ant_data, bandwidth_data, aco_data, solutions):
    """
    get_aco_data(origin, ant_data, bandwidth_data, aco_data, solutions)
    updates the aco_data and solutions global variables with the best and
    possible paths that have been discovered by the ants. This is done using
    the ant paths and bandwidth data and is specific for only the path specified
    by the user as the origin node.
    origin: the start node from which a best path was being sought by the user
    ant_data: the global variable containing paths followed by the ants
    bandwidth_data: data on the bandwiths between the various nodes in the
    search space
    aco_data: the gloabl variable containing best and possible paths for map visualization
    solutions: global variable containing the best path from user-specified node for
    updating the solutions file
    returns: aco_data and solutions variables with updates values
    """
    for ant, data in ant_data.items():
        if data['start_node'] == origin:
            for path in data['paths']:
                path_end_to_end_transmission = path['end-to-end_transmission']
                path_latency = path['latency']
                path_absolute_cost = path['cost']
                if path_absolute_cost > 0:
                    # update the map visualization data and best solution data for recording in the solutions file
                    if len(aco_data['best_path']) == 0:
                        # if there is no best solution yet, add this as one
                        solutions[0] = {"source node": f"Node-{path['path'][0]}"}
                        routing_path = ''
                        for idx, sensor in enumerate(path['path'][1:]):
                            routing_path += f"(Node-{sensor}, {bandwidth_data[path['path'][idx] - 1][path['path'][idx + 1] - 1]} Mbps),"
                        solutions[0]["routing path"] = routing_path
                        solutions[0]["end-to-end transmission rate"] = f"{path_end_to_end_transmission} Mbps"
                        solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                        solutions[0]["absolute path cost"] = f"{path_absolute_cost}"
                        aco_data['best_path'][str(path['path'])] = {'end_to_end_bandwidth': path_end_to_end_transmission,
                                                                         'path_latency': path_latency,
                                                                         'absolute_cost': path_absolute_cost
                                                                         }
                    else:
                        best_path = aco_data['best_path'][list(aco_data['best_path'].keys())[0]]
                        if path_absolute_cost > best_path['absolute_cost']:
                            '''
                            if current solution is better than best solution, add best solution as 
                            possible solution and set the current solution as the best solution
                            '''
                            solutions[0] = {"source node": f"Node-{path['path'][0]}"}
                            routing_path = ''
                            for idx, sensor in enumerate(path['path'][1:-1]):
                                routing_path += f"(Node-{sensor}, {bandwidth_data[idx][idx + 1]} Mbps),"
                            solutions[0]["routing path"] = routing_path
                            solutions[0]["end-to-end transmission rate"] = f"{path_end_to_end_transmission} Mbps"
                            solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                            solutions[0]["absolute path cost"] = f"{path_absolute_cost}"
                            aco_data['possible_paths'][list(aco_data['best_path'].keys())[0]] = {
                                                     'end_to_end_bandwidth': best_path['end_to_end_bandwidth'],
                                                     'path_latency': best_path['path_latency'],
                                                     'absolute_cost': best_path['absolute_cost']
                                                    }
                            aco_data['best_path'].pop(list(aco_data['best_path'].keys())[0])
                            aco_data['best_path'][str(path['path'])] = {
                                                     'end_to_end_bandwidth': path_end_to_end_transmission,
                                                     'path_latency': path_latency,
                                                     'absolute_cost': path_absolute_cost
                                                    }
                        else:
                            # if current solution is not better than best solution add it as possible solution
                            aco_data['possible_paths'][str(path['path'])] = {'end_to_end_bandwidth': path_end_to_end_transmission,
                                                                         'path_latency': path_latency,
                                                                         'absolute_cost': path_absolute_cost
                                                                         }

    return aco_data, solutions

In [849]:
def ant_colony_optimization(origin, Q=1, alpha = 0.6, evaporate=0.45, ants_count=20, max_iterations=10):
    """
    ant_colony_optimization(origin, Q=1, evaporate=0.45, ants_count=20, max_iterations=10)
    the ant colony optimization algorithm main method that gets the shortest path to 
    either of the base stations by combining all the functions in this module.
    origin: the start node (sensor) from which the search is being conducted
    Q (optional): weight used to reward a path that has been followed by an ant with pheromone (default: 1)
    alpha (optional): weight used to balance the significance of the pheromone level and path cost in determining
    the probability of an ant choosing a path (default: 0.6)
    evaporate (optional): the evaporation rate of the pheromone on the trail within a given 
    period of time (default: 0.45)
    ants_count (optional): the number of ants to explore the search space (default: 20)
    max_iterations (optional): the maximum number of times the ants explore the trail to get the best 
    paths to the destination (default: 10)
    returns: aco_data and solutions variables with updates values
    """
    # reinitialize all global variables every time the function is called
    trail_data = {}
    ant_data = {}
    aco_data = {'best_path': {},
            'possible_paths': {}}
    solutions = [{}]
    # initialize the ants
    init_ants(origin, x_y_data, ants_count, ant_data)
    # initialize pheromone randomly on the paths
    init_pheromone(x_y_data, trail_data)
    # calculate the bandwidth for every path
    bandwidth_data = calculate_route_costs(x_y_data)
    for i in range(max_iterations):
        # ants traverse through nodes from their respective origins to the destination
        traverse_nodes(ant_data, x_y_data, bandwidth_data, trail_data, alpha)
        # remove loops from the created trails
        remove_path_loops(ant_data)
        # calculate end-to-end cost for every followed path
        calculate_path_total_cost(ant_data, bandwidth_data)
        # evaporate the phromone for every path
        negative_feedback(evaporate, trail_data)
        # increase the pheromone for every followed path
        positive_feedback(Q, ant_data, trail_data, bandwidth_data)
    for ant, data in ant_data.items():
        print("ant {}: ({})".format(ant, data))
        print("--------------------------------------------------------")
    aco_data, solutions = get_aco_data(origin, ant_data, bandwidth_data, aco_data, solutions)
    return aco_data, solutions

In [879]:
#ant_colony_optimization(1)